In [1]:
from google.transit import gtfs_realtime_pb2
import requests

In [2]:
from datetime import datetime
import mysql.connector
from mysql.connector import Error
from mysql.connector import errorcode
from mysql.connector.constants import ClientFlag

In [6]:
feed = gtfs_realtime_pb2.FeedMessage()
response = requests.get('http://gtfs.ovapi.nl/nl/tripUpdates.pb')
rsn = '1'
rt = 1
all_tus = []
if(response.status_code == 200):
    feed.ParseFromString(response.content)
    ts = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(ts)
    for entity in feed.entity:
      if entity.HasField('trip_update'):
        if(entity.trip_update.trip.route_id == '57672'):
            print("entity.trip_update.trip.route_id --- ", entity.trip_update.trip.route_id)
            for x in entity.trip_update.stop_time_update:
                all_tus.append((entity.id, entity.trip_update.trip.route_id, rsn, rt, entity.trip_update.trip.trip_id, \
                       entity.trip_update.trip.direction_id, entity.trip_update.trip.start_date, entity.trip_update.trip.start_time,\
                       x.stop_sequence, x.stop_id, x.arrival.time, x.arrival.delay, x.departure.time, x.departure.delay, ts))
else:
    print("ERROR FETCHING REALTIME DATA FROM THE gtfs.ovapi.nl server")
    print(response.reason)

print(len(all_tus))

2019-01-22 10:12:04
entity.trip_update.trip.route_id ---  57672
entity.trip_update.trip.route_id ---  57672
entity.trip_update.trip.route_id ---  57672
entity.trip_update.trip.route_id ---  57672
entity.trip_update.trip.route_id ---  57672
entity.trip_update.trip.route_id ---  57672
159


In [74]:
# INSERT REALTIME DATA TO DB
def insert_rtdata(trip_updates):
    try:
        connection = mysql.connector.connect(host='localhost', database='gtfs', user='root', password='admin')
        cursor = connection.cursor()
        tu_query = """ INSERT INTO GTFS.trip_updates
                        (entity_id, route_id, route_short_name, route_type, trip_id, direction_id, start_date, start_time,
                        stop_sequence, stop_id, arrival_time, arrival_delay, departure_time, departure_delay, fetch_time)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"""
        # INCREASE PACKET SIZE TO BULK INSERT DATA
        packet_sql = 'SET GLOBAL max_allowed_packet=256*1024*1024'
        cursor.execute(packet_sql)
        result  = cursor.executemany(tu_query, trip_updates)
        connection.commit()
        print("Record inserted successfully into GTFS.trip_updates table ... ")
        
    except mysql.connector.Error as error :
        connection.rollback() #rollback if any exception occured
        print("Failed inserting record into GTFS.trip_updates table ... {}".format(error))

    finally:
        #closing database connection.
        if(connection.is_connected()):
            cursor.close()
            connection.close()
            print("MySQL connection is closed ... ")

In [75]:
insert_rtdata(all_tus)

Record inserted successfully into GTFS.trip_updates table ... 
MySQL connection is closed ... 


In [88]:
print(all_tus[350])

('2019-01-20:CXX:A006:7027', '1068', '1', 1, '85058488', 0, '20190120', '14:17:00', 9, '356115', 1547990986, 46, 1547991002, 62, '2019-01-20 14:46:37')
